In [1]:
import pandas as pd

from log_processing import process_log


In [2]:
log_file = "../backtests/newest.log"

trader_data, products_data, activities, trades = process_log(log_file)

In [6]:
products_data["RAINFOREST_RESIN"]["orders"]

0       [[-1, 10002], [15, 9997], [-15, 10003]]
1                    [[15, 9997], [-15, 10003]]
2                    [[15, 9996], [-15, 10004]]
3                    [[15, 9997], [-15, 10003]]
4                    [[15, 9997], [-15, 10003]]
                         ...                   
1995    [[-1, 10002], [15, 9997], [-15, 10003]]
1996                 [[15, 9997], [-15, 10001]]
1997                 [[15, 9999], [-15, 10003]]
1998                 [[15, 9996], [-15, 10000]]
1999                 [[15, 9997], [-15, 10003]]
Name: orders, Length: 2000, dtype: object